<a href="https://colab.research.google.com/github/AnshuYadav07/ExactInference_BayesianNetwork/blob/main/ExactInference_BayesianNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pgmpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.7 MB/s eta 0:00:00


In [7]:
# Import necessary libraries
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Step 1: Define the structure of the Bayesian Network
model = BayesianNetwork([('Disease', 'Fever'),
                         ('Disease', 'Cough'),
                         ('Disease', 'Fatigue')])


In [9]:

# Step 2: Define the Conditional Probability Distributions (CPDs)
# CPD for Disease (root node)
cpd_disease = TabularCPD(variable='Disease',
                         variable_card=2,
                         values=[[0.3], [0.7]],
                         state_names={'Disease': ['No Disease', 'Has Disease']})

# CPD for Fever given Disease
cpd_fever = TabularCPD(variable='Fever',
                       variable_card=2,
                       values=[[0.8, 0.2], [0.2, 0.8]],
                       evidence=['Disease'],
                       evidence_card=[2],
                       state_names={'Fever': ['No Fever', 'Has Fever'], 'Disease': ['No Disease', 'Has Disease']})

# CPD for Cough given Disease
cpd_cough = TabularCPD(variable='Cough',
                       variable_card=2,
                       values=[[0.7, 0.3], [0.3, 0.7]],
                       evidence=['Disease'],
                       evidence_card=[2],
                       state_names={'Cough': ['No Cough', 'Has Cough'], 'Disease': ['No Disease', 'Has Disease']})

# CPD for Fatigue given Disease
cpd_fatigue = TabularCPD(variable='Fatigue',
                         variable_card=2,
                         values=[[0.9, 0.4], [0.1, 0.6]],
                         evidence=['Disease'],
                         evidence_card=[2],
                         state_names={'Fatigue': ['No Fatigue', 'Has Fatigue'], 'Disease': ['No Disease', 'Has Disease']})




In [11]:
# Step 3: Add the CPDs to the model
model.add_cpds(cpd_disease, cpd_fever, cpd_cough, cpd_fatigue)



In [12]:
# Step 4: Verify the model
assert model.check_model()

In [13]:
# Step 5: Perform exact inference using Variable Elimination
infer = VariableElimination(model)


In [14]:
# Step 6: Define the symptoms provided by the user as evidence
# Example: the user has Fever and Cough but no Fatigue
evidence = {'Fever': 'Has Fever', 'Cough': 'Has Cough', 'Fatigue': 'No Fatigue'}

# Step 7: Query the probability of having the disease given the symptoms
result = infer.query(variables=['Disease'], evidence=evidence)

# Step 8: Print the result
print(result)

+----------------------+----------------+
| Disease              |   phi(Disease) |
+======================+================+
| Disease(No Disease)  |         0.0936 |
+----------------------+----------------+
| Disease(Has Disease) |         0.9064 |
+----------------------+----------------+


In [5]:
# Step 8: Create a Markov Network for Symptom Correlations
# Create a Markov Network (Markov Random Field)
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference import BeliefPropagation

markov_net = MarkovNetwork()

# Define the structure (Undirected Graph)
markov_net.add_edges_from([('Fever', 'Cough'), ('Disease', 'Fever'), ('Disease', 'Cough')])

# Add potential functions (factors) to the Markov Network
# These can be thought of as joint probability distributions over the connected nodes
factor_disease_fever = DiscreteFactor(variables=['Disease', 'Fever'], cardinality=[2, 2],
                                      values=[0.8, 0.2, 0.1, 0.9])  # P(Fever | Disease)

factor_disease_cough = DiscreteFactor(variables=['Disease', 'Cough'], cardinality=[2, 2],
                                      values=[0.9, 0.1, 0.4, 0.6])  # P(Cough | Disease)

factor_fever_cough = DiscreteFactor(variables=['Fever', 'Cough'], cardinality=[2, 2],
                                    values=[0.95, 0.05, 0.5, 0.5])  # Correlation between Fever & Cough

# Add the factors to the Markov Network
markov_net.add_factors(factor_disease_fever, factor_disease_cough, factor_fever_cough)

# Perform inference on the Markov Network using Belief Propagation
belief_propagation = BeliefPropagation(markov_net)

# Query: What is the marginal probability distribution of Fever and Cough?
marginal_fever = belief_propagation.query(variables=['Fever'])
marginal_cough = belief_propagation.query(variables=['Cough'])

print("\nMarkov Network Inference Results:")
print("Marginal Probability of Fever:")
print(marginal_fever)


Markov Network Inference Results:
Marginal Probability of Fever:
+----------+--------------+
| Fever    |   phi(Fever) |
+==========+==============+
| Fever(0) |       0.5700 |
+----------+--------------+
| Fever(1) |       0.4300 |
+----------+--------------+


In [6]:
print("\nMarginal Probability of Cough:")
print(marginal_cough)


Marginal Probability of Cough:
+----------+--------------+
| Cough    |   phi(Cough) |
+==========+==============+
| Cough(0) |       0.7756 |
+----------+--------------+
| Cough(1) |       0.2244 |
+----------+--------------+
